# Mathias Henry Morgan

## 35087056

## Assessment 2: Big data project - Phase 2

## Data Cleaning and Wrangling Pipeline

## Instructions

1. ***Run all*** to import libraries, define function logic and trigger the execute_transform_pipeline() function
2. The cell block containing execute_transform_pipeline() will contain logging statements for visibility of actions performed
3. At the completion of the run a cleaned file will be generated with a unique date time stamp

In [43]:
# Import libraries
import pandas as pd
from sklearn import preprocessing
from pathlib import Path
import uuid
import warnings
import datetime
warnings.filterwarnings('ignore') # Removes warning messages from print logging

# Raw Meta Data

In [44]:
def meta_data_info(churnDataFrame: pd.DataFrame) -> None:
    """
    Prints metadata of raw churn data set.
    :param churnDataFrame: churn data set.
    :returns: None.
    """
    try:
        # Print relevant meta data
        print(f"Column Count: {len(churnDataFrame.columns)}")
        print(f"Row Count: {len(churnDataFrame)}")
        print(churnDataFrame.info())
    except Exception as e:
        print(f"Error while printing metadata: {e}")

# Remove Errors

In [45]:
def remove_errors(churnDataFrame: pd.DataFrame) -> pd.DataFrame:
    """
    Removes rows where 'avg_frequency_login_days' contains the string 'Error'.
    :param churnDataFrame: DataFrame containing churn data.
    :return: Processed DataFrame with error rows removed.
    """
    try:
        rowCount = len(churnDataFrame)
        errorCounts = len(churnDataFrame[churnDataFrame["avg_frequency_login_days"] == "Error"])
        churnDataFrame = churnDataFrame[churnDataFrame["avg_frequency_login_days"] != "Error"]
        print(f'{errorCounts} Errors Rows Removed out of {rowCount}')
    except Exception as e:
        print(f"Error while removing rows with 'Error' in 'avg_frequency_login_days': {e}")
    return churnDataFrame

# Remove Duplicates 

In [46]:
def remove_duplicates(churnDataFrame: pd.DataFrame) -> pd.DataFrame:
    """
    Removes rows with duplicate 'security_no' values, keeping the first occurrence.
    :param churnDataFrame: churn data set.
    :returns: processed churn datas set with duplicate rows removed if present.
    """
    try:
        originalSize = len(churnDataFrame)
        churnDataFrame = churnDataFrame.drop_duplicates(subset="security_no", keep="first") # Remove duplicates if present, keep first record if duplicated
        NewSize = len(churnDataFrame)
        numDuplicates = originalSize - NewSize
        print(f"Number of duplicates removed: {numDuplicates}") # Print number of duplicates present
    except Exception as e:
        print(f"Error during duplicate removal: {e}")
    return churnDataFrame

# Churn Risk Distribution

In [47]:
def churn_distrubtion(churnDataFrame: pd.DataFrame) -> None:
    """
    Displays the percentage of customers at risk and not at risk of churn,
    based on the 'churn_risk_score' column (if present).
    :param churnDataFrame: churn data set.
    :returns: None.
    """
    try:
        cols = churnDataFrame.columns
        if "churn_risk_score" in cols: # Checks if 'churn_risk_score' is present
            print((churnDataFrame["churn_risk_score"].value_counts( # Print dataframe of churn risk distribution
                normalize=True) * 100).round(2).to_frame().T.rename(columns={1:"Percentage At Risk", 0:"Percentage Not At Risk"}))
            churnDataFrame["churn_risk_score"] = churnDataFrame["churn_risk_score"].astype("Int64")
        else:
            print("'churn_risk_score' field not present in data set")
    except Exception as e:
        print(f"Error while calculating churn distribution: {e}")

# Numeric Field Transformations

In [48]:
def numeric_transform(churnDataFrame: pd.DataFrame) -> pd.DataFrame:
    """
    Detects and handles extreme outliers in specified integer and float columns of the churn dataset
    using the Interquartile Range (IQR) method. Far outliers are considered invalid records and are
    replaced with the median of the respective column (excluding the outliers). All values are converted
    to absolute, numeric types and rounded—integers to whole numbers, floats to two decimal places.
    :param churnDataFrame: DataFrame containing churn data.
    :return: Processed DataFrame with erroneous outliers imputed with the median and rounded.
    """
    intColsTransform = ["days_since_last_login", "age", "avg_frequency_login_days"]
    floatColsTransform = ["avg_time_spent", "avg_transaction_value", "points_in_wallet"]

    for col in intColsTransform:
        try:
            print(f"Processing Integer values in '{col}'...")
            churnDataFrame[col] = abs(pd.to_numeric(churnDataFrame[col], errors="coerce")).round(0).astype('Int64')
            Q1 = churnDataFrame[col].quantile(0.10)
            Q3 = churnDataFrame[col].quantile(0.90)
            IQR = Q3 - Q1
            lower_bound = Q1 - 4 * IQR
            upper_bound = Q3 + 4 * IQR
            intOutliers = churnDataFrame[col][(churnDataFrame[col] < lower_bound) | (churnDataFrame[col] > upper_bound)]
            if intOutliers.empty:
                print(f"No far outliers found in {col}...")
            else:
                print(f"{len(intOutliers)} far outliers found in {col}: {sorted(set(intOutliers))}...")
                medianVal = churnDataFrame[col].loc[~churnDataFrame.index.isin(intOutliers.index)].median()
                churnDataFrame.loc[churnDataFrame.index.isin(intOutliers.index), col] = medianVal
                print(f"The median value of {medianVal} imputed for all outliers")
        except Exception as e:
            print(f"Error processing integer column '{col}': {e}")

    for col in floatColsTransform:
        try:
            print(f"Processing float values in '{col}'...")
            churnDataFrame[col] = abs(pd.to_numeric(churnDataFrame[col], errors="coerce")).round(2).astype("float64")
            Q1 = churnDataFrame[col].quantile(0.10)
            Q3 = churnDataFrame[col].quantile(0.90)
            IQR = Q3 - Q1
            lower_bound = Q1 - 4 * IQR
            upper_bound = Q3 + 4 * IQR
            floatOutliers = churnDataFrame[col][(churnDataFrame[col] < lower_bound) | (churnDataFrame[col] > upper_bound)]
            if floatOutliers.empty:
                print(f"No far outliers found in {col}...")
            else:
                print(f"{len(floatOutliers)} far outliers found in {col}: {sorted(set(floatOutliers))}...")
                medianVal = churnDataFrame[col].loc[~churnDataFrame.index.isin(floatOutliers.index)].median()
                churnDataFrame.loc[churnDataFrame.index.isin(floatOutliers.index), col] = medianVal
                print(f"The median value of {medianVal} imputed for all outliers")
        except Exception as e:
            print(f"Error processing float column '{col}': {e}")
    return churnDataFrame

# Process Null Values

In [49]:
def process_nulls(churnDataFrame: pd.DataFrame) -> pd.DataFrame:
    """
    Identifies columns with null values and imputes missing data based on the column's data type.
    String columns are imputed with 'Unknown', numeric columns with 0 (rounded to two decimals), and
    boolean-like columns with 'No'. Provides visibility into null counts before and after processing.
    :param churnDataFrame: DataFrame containing churn data.
    :return: Processed DataFrame with null values imputed based on field type.
    """
    try:
        print("Identify columns containing null records...")
        missingDataCols = list(churnDataFrame.isna().sum()[churnDataFrame.isna().sum() > 0].to_frame().T.columns)
        for col in missingDataCols:
            print(f"{col} contains: {churnDataFrame[col].isna().sum()} null values")
        boolCols = ["used_special_discount", "offer_application_preference", "past_complaint"]
        stringCols = [col for col in list(churnDataFrame.select_dtypes(include="object").columns) if col in missingDataCols and col not in boolCols]
        numericCols = [col for col in list(churnDataFrame.select_dtypes(include=["int64", "float64"]).columns) if col in missingDataCols]

        print()
        print("Processing null values in string columns...")
        for col in stringCols:
            colNullCount = churnDataFrame[col].isna().sum()
            print(f"Processing {colNullCount} null records in {col}...")
            churnDataFrame[col] = churnDataFrame[col].apply(lambda row: "Unknown" if pd.isnull(row) else row)

        print()
        print("Processing null values in numeric columns...")
        for col in numericCols:
            colNullCount = churnDataFrame[col].isna().sum()
            print(f"Processing {colNullCount} null records in {col}...")
            churnDataFrame[col] = churnDataFrame[col].apply(lambda row: 0 if pd.isnull(row) else row).round(2)

        print()
        print("Processing null values in boolean columns...")
        for col in boolCols:
            if col in churnDataFrame.columns:
                colNullCount = churnDataFrame[col].isna().sum()
                print(f"Processing {colNullCount} null records in {col}...")
                churnDataFrame[col] = churnDataFrame[col].apply(lambda row: 'No' if pd.isnull(row) else row)

        print()
        print("Null records processed...")
    except Exception as e:
        print(f"Error while processing null values: {e}")

    return churnDataFrame

# Process Date and Time Fields

In [50]:
def date_transformation(churnDataFrame: pd.DataFrame) -> pd.DataFrame:
    """
    Converts the 'joining_date' field to datetime format and splits it into separate
    year, month, and day columns.
    The original 'joining_date' column, which contains date values in the format
    Year-Month-Day, is removed after transformation.
    :param churnDataFrame: DataFrame containing churn data.
    :return: Processed DataFrame with date fields split into 'join_year', 'join_month', and 'join_day'.
    """
    try:
        churnDataFrame["joining_date"] = pd.to_datetime(churnDataFrame["joining_date"])
        churnDataFrame["join_year"] = churnDataFrame["joining_date"].dt.year.astype("Int64")
        churnDataFrame["join_month"] = churnDataFrame["joining_date"].dt.month.astype("Int64")
        churnDataFrame["join_day"] = churnDataFrame["joining_date"].dt.day.astype("Int64")
        churnDataFrame = churnDataFrame.drop(columns=["joining_date"])
        print("joining_date field split into: 'join_year', 'join_month', 'join_day'...")
    except Exception as e:
        print(f"Error in date transformation: {e}")
    return churnDataFrame

def time_transformation(churnDataFrame: pd.DataFrame) -> pd.DataFrame:
    """
    Converts the 'last_visit_time' field to datetime format (HH:MM:SS) and splits it into
    hour, minute, and second columns.
    The original 'last_visit_time' column is removed after transformation.
    :param churnDataFrame: DataFrame containing churn data.
    :return: Processed DataFrame with time fields split into 'last_visit_hour', 'last_visit_min', and 'last_visit_sec'.
    """
    try:
        churnDataFrame["last_visit_time"] = pd.to_datetime(churnDataFrame["last_visit_time"], format="%H:%M:%S")
        churnDataFrame["last_visit_hour"] = churnDataFrame["last_visit_time"].dt.hour.astype("Int64")
        churnDataFrame["last_visit_min"] = churnDataFrame["last_visit_time"].dt.minute.astype("Int64")
        churnDataFrame["last_visit_sec"] = churnDataFrame["last_visit_time"].dt.second.astype("Int64")
        churnDataFrame = churnDataFrame.drop(columns=["last_visit_time"])
        print("last_visit_time field split into: 'last_visit_hour', 'last_visit_min', 'last_visit_sec'...")
    except Exception as e:
        print(f"Error in time transformation: {e}")
    return churnDataFrame

# Process Special Character Values

In [51]:
def special_characters(churnDataFrame: pd.DataFrame) -> pd.DataFrame:
    """
    Identifies and handles special characters (e.g., !, ?, #) in column values,
    as they are considered erroneous. Replaces such values with appropriate substitutes.
    The 'joined_through_referral' column is handled separately based on 'referral_id' logic.
    :returns: processed churn data set with errenous special characters removed and values imputed.
    """
    try:
        specialChars = r"[!?#]" # Define special characters to find
        colsSpecialChar = [
            col for col in churnDataFrame.columns
            if churnDataFrame[col].astype(str).str.contains(specialChars, regex=True).any()] # List comprehension of columns containing special character regex pattern
        print(f"Columns containing special character values: {colsSpecialChar}")

        # Loop over target columns
        for col in colsSpecialChar:
            uniqueVals = list(set(churnDataFrame[col].to_list())) # Get list of unique special characters in column
            SpecialCharCount = churnDataFrame[col].str.count(specialChars).sum()
            if col in "joined_through_referral": # Handle this field specifically, if ID is present can assume Yes else No
                churnDataFrame[col] = churnDataFrame.apply(
                    lambda row: "No" if row["referral_id"] in ["xxxxxxxx"] and row["joined_through_referral"] in uniqueVals else "Yes",
                    axis=1) # Row level imputation
            else:
                churnDataFrame[col] = churnDataFrame[col].apply(lambda row: "Unknown" if row in specialChars else row) # Otherwise impute calue at row level
            print(f"Processed {SpecialCharCount} records of special charcters in '{col}' column.....")
    except Exception as e:
        print(f"Error in special character handling: {e}")
    return churnDataFrame

# Sentiment Mapping of Feedback

In [52]:
def feedback_transform(churnDataFrame: pd.DataFrame) -> pd.DataFrame:
    """
    Transforms the 'feedback' column into sentiment classifications using a predefined mapping:
    0 = Negative, 1 = Positive, 2 = Neutral.
    This transformation uses hardcoded mappings based on known feedback values.
    Trailing spaces are stripped before applying the mapping to ensure consistent classification.
    :param churnDataFrame: churn data set.
    :returns: processed churn data set with feedback values mapped to pre defined integers.
    """
    try:
        # Transforming feedback values into sentiment classification
        sentimentMapping = {
            "Products always in Stock":1,
            "User Friendly Website":1,
            "Poor Customer Service":0,
            "Poor Product Quality":0,
            "Reasonable Price":1,
            "Quality Customer Care":1,
            "Too many ads":0,
            "Poor Website":0,
            "No reason specified":2}
        # Loop over mappings dictionary and display
        for k,v in sentimentMapping.items():
            print(f"Mapping value: '{k}', to integer value: {v}")
        churnDataFrame["feedback"] = churnDataFrame["feedback"].str.strip() # Ensure no trailing spaces are present in the field to allow for proper value conversion
        churnDataFrame["feedback"] = churnDataFrame["feedback"].apply(lambda row: sentimentMapping[row] if row in sentimentMapping else row) # Alter values using row level imputation on field
        churnDataFrame["feedback"] = churnDataFrame["feedback"].astype('Int64')
    except Exception as e:
        print(f"Error in feedback transformation: {e}")
    return churnDataFrame

# Label Encode Categorical Values

In [53]:
def label_encode(churnDataFrame: pd.DataFrame) -> pd.DataFrame:
    """
    Performs label encoding on all categorical columns, converting string values
    into integers for compatibility with tree-based models. The value mappings for each column
    are displayed in the logs.
    :param churnDataFrame: churn data set.
    :returns: processed churn data set with categorical variables conerted to integers
    """
    try:
        # Identify categorical fields and loop over them to convert categorical values
        categoricalCols = list(churnDataFrame.select_dtypes(include='object').columns)
        for col in categoricalCols:
            print(f"Label Encoding values in {col}...")
            categoricalVals = list(set(churnDataFrame[col])) # Get a list of all necessary columns
            labelEncoder = preprocessing.LabelEncoder() # Define Label Encoder from
            churnDataFrame[col] = labelEncoder.fit_transform(churnDataFrame[col]) # Apply the encoding to target field
            churnDataFrame[col] = churnDataFrame[col].astype("Int64")
            print(f"Categorical values mapped from {categoricalVals} to integer values: {list(set(churnDataFrame[col]))}")
    except Exception as e:
        print(f"Error in label encoding: {e}")
    return churnDataFrame

# Remove unwanted features

In [54]:
def remove_features(churnDataFrame: pd.DataFrame) -> pd.DataFrame:
    """
    Removes the 'security_no' and 'referral_id' columns from the dataset and
    reorders the 'churn_risk_score' column to appear last, if present.
    :param churnDataFrame: churn data set.
    :returns: processed churn data set with certain fields removed and columns re-ordered.
    """
    try:
        # Remove unnecessary fields
        churnDataFrame = churnDataFrame.drop(columns=["security_no","referral_id"])
        print("Removed 'security_no' and 'referral_id' columns...")
        cols = [col for col in churnDataFrame.columns if col not in ["churn_risk_score"]] # Get a list of call fields without churn_risk_score
        if "churn_risk_score" in churnDataFrame.columns:
            newCols = cols + ["churn_risk_score"] # Add churn_risk_score to the end if present
            churnDataFrame = churnDataFrame.reindex(columns=newCols) # Alter column order ensuring churn_risk_score is the last field in data frame reading left to right
            print("Re-Indexed 'churn_risk_score' field")
    except Exception as e:
        print(f"Error in feature removal: {e}")
    return churnDataFrame

# Execute Pipeline Logic

In [55]:
def execute_transform_pipeline() -> None:
    """
    Executes the full data cleaning and transformation pipeline for churn data.
    This function processes all raw churn CSV files in the current directory that match
    the pattern 'churn_raw*.csv'. It sequentially applies a series of transformation
    steps, including error removal, deduplication, null imputation, encoding, and more.
    For each file, a unique run ID is generated and logged. The cleaned dataset is
    exported as a new CSV file with a timestamped filename.
    :returns: None
    """
    # Extract the raw data csv file to be processed
    csvFiles = [str(file) for file in Path(".").glob("churn_raw*.csv")] # Get a list of all CSV files with the name churn_raw.csv OR and extension e.g churn_raw_1.csv
    for fileName in csvFiles:
        try:
            churnData = pd.read_csv(fileName)

            # Creating random ID to simulate a real data transformation pipeline in log output
            randomId = uuid.uuid4()
            stringID = str(randomId)
            print(f"Executing Data Transformation Pipeline...")
            print() # Creates a gap for for clean logging output
            print(f"Run ID: {stringID}")
            print()

            print("----- Raw Churn Metadata -----",end="\n")
            meta_data_info(churnData)
            print()

            print("----- Removing Errenous Rows -----",end="\n")
            churnData = remove_errors(churnData)
            print()

            print("----- Removing Duplicate Rows -----",end="\n")
            churnData = remove_duplicates(churnData)
            print()

            print("----- Distribution (%) of Customers at risk of Churn -----",end="\n")
            churn_distrubtion(churnData)
            print()

            print("----- Transform Numeric Values -----",end="\n")
            churnData = numeric_transform(churnData)
            print()

            print("----- Impute Values for Null Records -----",end="\n")
            churnData = process_nulls(churnData)
            print()

            print("----- Transform Date and Time Fields -----",end="\n")
            churnData = date_transformation(churnData)
            churnData = time_transformation(churnData)
            print()

            print("----- Impute Values for Special Characters -----",end="\n")
            churnData = special_characters(churnData)
            print()

            print("----- Transform Feedback Column -----",end="\n")
            churnData = feedback_transform(churnData)
            print()

            print("----- Removing Unnecessary Features -----",end="\n")
            churnData = remove_features(churnData)
            print()

            print("----- Encode Categorical Variables -----",end="\n")
            churnData = label_encode(churnData)
            print()

            print("----- Processed Churn Metadata -----",end="\n")
            if len(churnData) > 20000:
                churnData = churnData.iloc[:20000] # Limit row count to 20,000 rows
            meta_data_info(churnData)
            print()

            print(f"Data Transformation Run ID: {stringID} Complete...")

            currentTime = datetime.datetime.now() # Get current date and time
            fileName = f"churn_clean_{currentTime}.csv" # Define cleaned file name with unique date time stamp
            cleanFolderPath = f"{fileName}"
            churnData.to_csv(cleanFolderPath, index=False) # Write out cleaned CSV file to folder
            print(f"Transformed data set: {fileName} written to main folder...")
        except Exception as e:
            print(f"Error processing raw file '{fileName}'': {e}")

# Execute Pipeline

In [56]:
# Execute transformation pipeline
execute_transform_pipeline()

Executing Data Transformation Pipeline...

Run ID: a40cfddc-d551-4ddb-a799-d05bd8e4cd85

----- Raw Churn Metadata -----
Column Count: 23
Row Count: 36992
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           36992 non-null  int64  
 1   gender                        36992 non-null  object 
 2   security_no                   36992 non-null  object 
 3   region_category               31564 non-null  object 
 4   membership_category           36992 non-null  object 
 5   joining_date                  36992 non-null  object 
 6   joined_through_referral       36992 non-null  object 
 7   referral_id                   36992 non-null  object 
 8   preferred_offer_types         36704 non-null  object 
 9   medium_of_operation           36992 non-null  object 
 10  internet_option         

# References

GeeksforGeeks. (n.d.). Change the order of a Pandas DataFrame columns in Python. https://www.geeksforgeeks.org/change-the-order-of-a-pandas-dataframe-columns-in-python/

GeeksforGeeks. (2025, January 2). Detect and remove the outliers using Python. https://www.geeksforgeeks.org/detect-and-remove-the-outliers-using-python/

GeeksforGeeks. (n.d.). Generating random IDs using UUID in Python. https://www.geeksforgeeks.org/generating-random-ids-using-uuid-python/

GeeksforGeeks. (n.d.). Get current timestamp using Python. https://www.geeksforgeeks.org/get-current-timestamp-using-python/

GeeksforGeeks. (2025, February 12). Label encoding in Python. https://www.geeksforgeeks.org/ml-label-encoding-of-datasets-in-python/

nkmk. (n.d.). pandas: Select columns by dtype with select_dtypes(). nkmk note. https://note.nkmk.me/en/python-pandas-select-dtypes/

Stack Overflow. (n.d.). Change column type in pandas. Stack Exchange. https://stackoverflow.com/questions/15891038/change-column-type-in-pandas

Stack Overflow. (n.d.). Hide all warnings in IPython. Stack Exchange. https://stackoverflow.com/questions/9031783/hide-all-warnings-in-ipython

Stack Overflow. (n.d.). Splitting timestamp column into separate date and time columns. Stack Exchange. https://stackoverflow.com/questions/35595710/splitting-timestamp-column-into-separate-date-and-time-columns

Stack Overflow. (n.d.). Using a dictionary of lambda functions in pandas.assign() gives the wrong result. How to avoid the lazy binding? Stack Exchange. https://stackoverflow.com/questions/72544610/using-a-dictionary-of-lambda-functions-in-pandas-assign-gives-the-wrong-result